In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up directories
base_dir = "bengali_digits"
train_dir = base_dir

# Using ImageDataGenerator to read images from directories
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalize and split data

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=20,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical',
    subset='validation'
)


Found 3747 images belonging to 3 classes.
Found 935 images belonging to 3 classes.


In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 digits
])

In [7]:
steps_per_epoch = len(train_generator)  
validation_steps = len(validation_generator)

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_steps
)





Epoch 1/50
188/188 [==============================] - 8s 39ms/step - loss: 0.0127 - accuracy: 0.9973 - val_loss: 0.1498 - val_accuracy: 0.9786
Epoch 2/50
188/188 [==============================] - 7s 36ms/step - loss: 0.0158 - accuracy: 0.9957 - val_loss: 0.1031 - val_accuracy: 0.9818
Epoch 3/50
188/188 [==============================] - 7s 35ms/step - loss: 0.0038 - accuracy: 0.9984 - val_loss: 0.2149 - val_accuracy: 0.9722
Epoch 4/50
188/188 [==============================] - 7s 35ms/step - loss: 0.0095 - accuracy: 0.9976 - val_loss: 0.1087 - val_accuracy: 0.9775
Epoch 5/50
188/188 [==============================] - 7s 36ms/step - loss: 0.0120 - accuracy: 0.9968 - val_loss: 0.3072 - val_accuracy: 0.9711
Epoch 6/50
188/188 [==============================] - 7s 35ms/step - loss: 0.0178 - accuracy: 0.9957 - val_loss: 0.0812 - val_accuracy: 0.9797
Epoch 7/50
188/188 [==============================] - 7s 35ms/step - loss: 0.0055 - accuracy: 0.9989 - val_loss: 0.0564 - val_accuracy: 0.9840

In [9]:
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


47/47 [==============================] - 1s 22ms/step - loss: 0.0729 - accuracy: 0.9893
Validation Accuracy: 98.93%


In [17]:
model.save("digit_classify.keras")

In [19]:
import numpy as np
from tensorflow.keras.preprocessing import image
import cv2
from tensorflow.keras.models import load_model
loaded_model = load_model("digit_classify.keras")

class_map = {
    0: 'zero',
    1: 'one',
    2: 'two'
}

img_path = 'test1.png'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array_norm = np.expand_dims(img_array, axis=0) / 255.0  # Normalize
predictions = loaded_model.predict(img_array_norm)
print(predictions)
predicted_class = np.argmax(predictions[0])
predicted_label = class_map[predicted_class]
img_array_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)  # Convert to BGR for OpenCV
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img_array_bgr, predicted_label, (10, 30), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
output_path = 'testpredict.jpeg'  # Replace with where you want to save the image
cv2.imwrite(output_path, img_array_bgr)

1/1 [==============================] - 0s 94ms/step
[[3.1908184e-13 7.2966579e-09 1.0000000e+00]]


True